In [57]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pathlib
import zipfile
import torch
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cpu


In [4]:
def totalSize(path):
  sum = 0;
  for path, subdirs, files in os.walk(path):  ## walk directory
    sum += len(files)
    # for name in files:
        # print(os.path.join(path, name))
  return sum

In [8]:
base_dir='D:\Project\Machine Learning Projects\Datasets\Hands'

In [9]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [10]:
print("Train Data Size : ",totalSize(train_dir))
print("Validation Data Size : ", totalSize(validation_dir))

Train Data Size :  2118
Validation Data Size :  999


In [11]:
classes = os.listdir(train_dir) 
classes

['1', '2', '3', '4', '5']

In [12]:
BATCH_SIZE = 32 ## number of image batch size 
IMAGE_SIZE = 128 ## set image height width to 128x128
transform = transforms.Compose([
                                # transforms.ToPILImage(),
                                # transforms.Grayscale(),
                                transforms.RandomRotation(20,expand=True), ## adding random rotation 20deg
                                torchvision.transforms.ColorJitter(hue=.05, saturation=.05), ## adding color filter
                                transforms.RandomVerticalFlip(), ## adding vertical flip
                                transforms.RandomHorizontalFlip(), ## adding horizontal flip
                                transforms.Resize(IMAGE_SIZE),  ## image resize
                                transforms.CenterCrop(IMAGE_SIZE), ## image center crop
                                transforms.ToTensor(), ## array converted into torch tensor and then divided by 255 (1.0/255)
                                # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

In [13]:
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_dir,transform=transform),
    batch_size=32, shuffle=True
)

In [14]:
validation_loader=DataLoader(
    torchvision.datasets.ImageFolder(validation_dir,transform=transform),
    batch_size=32, shuffle=True
)

In [15]:
data_iter = iter(train_loader)
images, labels = data_iter.next()

In [42]:
print(images.size())  #[batch_size,color_channel,hight,width]

print(labels.size()) #one iter has 32 images and labels because batch size are 64 

torch.Size([32, 3, 128, 128])
torch.Size([32])


In [45]:
layers = [
    torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5), 
    torch.nn.MaxPool2d(2,2),
    torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5), 
    torch.nn.MaxPool2d(2,2),
    torch.nn.Conv2d(in_channels=32, out_channels=128, kernel_size=5),
    torch.nn.MaxPool2d(2,2),

]
 


In [46]:
x = images
print(x.shape,end='\n')

torch.Size([32, 3, 128, 128])


In [47]:
for layer in layers:
    output = layer(x)
    x = output
    print(x.shape)

torch.Size([32, 16, 124, 124])
torch.Size([32, 16, 62, 62])
torch.Size([32, 32, 58, 58])
torch.Size([32, 32, 29, 29])
torch.Size([32, 128, 25, 25])
torch.Size([32, 128, 12, 12])


In [54]:

class CNN(torch.nn.Module): 
    def __init__(self):
        super(CNN, self).__init__()

        #initializing 3 convolution layer
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.conv3 =  torch.nn.Conv2d(in_channels=32, out_channels=128, kernel_size=5)
        
        #initializing dropout 
        self.dropout = torch.nn.Dropout(0.2)
        
        self.pool= torch.nn.MaxPool2d(2,2)

 
        #initializing linear 
        self.fc1 = torch.nn.Linear(128 * 12 * 12, 512)
        self.fc2 = torch.nn.Linear(512, 5)

 
    def forward(self, x):
        
        
        x=x
        
        x=self.conv1(x)
        x=F.relu(x)
        x = self.pool(x)
        
        x=self.conv2(x)
        x=F.relu(x)
        x = self.pool(x) 

        x=self.conv3(x)
        x=F.relu(x)
        x = self.pool(x)
        
        x = x.view(-1, 128*12*12) 
        
        x = F.relu(self.fc1(x))
        
        x = self.fc2(x) 
        return x
        
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
model = CNN().to(device)
print(model)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=18432, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
)


In [55]:
num_classes = 5
batch_size = BATCH_SIZE
learning_rate = 0.001
 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

#  Training

In [62]:
epochs = 5
for epoch in range(epochs):
    
    for inputs, labels in tqdm(train_loader):
        
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        test = model(inputs)
        loss = criterion(test, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|                                                                                           | 0/67 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.1333073377609253


  0%|                                                                                           | 0/67 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.2768489122390747


  0%|                                                                                           | 0/67 [00:00<?, ?it/s]

Epoch: 2. Loss: 1.0356236696243286


  0%|                                                                                           | 0/67 [00:00<?, ?it/s]

Epoch: 3. Loss: 1.0026482343673706


100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:42<00:00,  1.56it/s]

Epoch: 4. Loss: 0.1259385198354721


In [64]:
PATH = './Hand_Finger_v2.pth'
torch.save(model.state_dict(), PATH)

In [67]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        pred = torch.argmax(logps, dim=1)
        correct = pred.eq(labels)
        

In [68]:
print(accuracy)

tensor(25.2946)
